# A simple liner regression example in Flux

In this notebook, we go through a simple linear regression model using [Flux](https://fluxml.ai/) which is a Machine Learning library for Julia. This example is based on [this one](https://discourse.julialang.org/t/training-a-simple-linear-model-in-flux/24741/2).

To train a ML model we need to define the following elements:

* Data set
* Parameters of the model
* Objective function
* Optimization routine

In [9]:
#Load the necessary packages
using Flux, Plots
using Flux: throttle, @epochs

#Backed for Plots
plotly()

┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots /home/liliana/.julia/packages/Plots/8GUYs/src/backends.jl:363


Plots.PlotlyBackend()

## Data set

Usually, we start by obtaining the data set that we use to train our model and test it. For this example, we'll create some random data and reshape it so we can pass it to our model.

We are creating a simple regression model with one input. Hence, we need to create a data set with one feature.

In [3]:
X = rand(100);
Y = 0.5X + rand(100);

We visualize the data we just created.

In [10]:
scatter(X, Y, title = "Data")

<!DOCTYPE html>
 
 
 Plots.jl

The data we just created might not have a strong linear correlation but still is useful for trying Flux.

Now, we split the data into train and test and prepare it for feeding it into the model.

In [20]:
train_x, train_y = reduce(hcat, X[1:80]), reduce(hcat, Y[1:80]);
test_x, test_y = reduce(hcat, X[81:100]), reduce(hcat, Y[81:100]);

**Note:** We need to split the data randomly.

In [23]:
#Zip the data
train_data = [(train_x, train_y)];

## Model

A simple linear regresssion model is defined as $m = \sigma.(W*x + b)$. We can define by setting a neuron with no activation function. In Flux, we can use the [Dense](https://fluxml.ai/Flux.jl/stable/models/layers/#Flux.Dense) function to define this model:

In [24]:
m = Dense(1, 1)

Dense(1, 1)

Also, we need to obtain the parameters of the model so we can pass it to the train function we'll define below.

In [25]:
ps = Flux.params(m)

Params([Float32[-1.3009212], Float32[0.0]])

When we create a model, Flux initializes the parameters with random values. At this point we obtain predictions with the current values for the parameters so that we can compare with the values obtained after training the model.

In [28]:
pred_0 = m(reduce(hcat,X));

## Loss function

For this example, we use the **MSE** function. Flux has many loss functions that we can use out of the box.

In [37]:
loss(x, y) = Flux.Losses.mse(m(x), y)

loss (generic function with 1 method)

## Optimiser

We need to set the optimization routine (optimiser) that we'll use to train our model. This optimiser will optimise the loss function.

In [30]:
#We use Gradient Descent with a learning rate of 0.3
opt = Descent(0.3)

Descent(0.3)

## Train the model

After setting the data, model, loss function and optimiser we can finally train our model. In Flux, we can execute a training step with the `Flux.train!` function.

In [38]:
Flux.train!(loss, ps, train_data, opt)

LoadError: DimensionMismatch("second dimension of A, 1, does not match length of x, 80")

Flux allows us to create a *callback* function so that we can print information during the training process.

In [39]:
evalcb() = @show(loss(test_x, test_y))

evalcb (generic function with 1 method)

In [40]:
Flux.train!(loss, ps, data, opt, cb = throttle(evalcb, 2))

LoadError: UndefVarError: data not defined

To train the model, we can either put the `Flux.train!` function inside a *for loop* or use the *@epoch* macro.